In [ ]:
#Downloads
!pip install -q gradio


In [ ]:
!pip install openai==0.28

In [ ]:
!pip install pandas gradio openpyxl

In [18]:
import gradio as gr
import pandas as pd
import openai

# Simulated user database
users = {
    "admin": {"password": "admin123", "role": "admin"},
    "user": {"password": "user123", "role": "user"}
}

# Dictionary to store data for each year
year_data = {str(year): pd.DataFrame() for year in range(2021, 2026)}


# Access code for registration
ACCESS_CODE = "123"

# Function to register a new user
def register_user(username, password, confirm_password):
    # Check if username already exists
    if username in users:
        return "Error: Username already exists.", gr.update(visible=True)

    # Check if passwords match
    if password != confirm_password:
        return "Error: Passwords do not match.", gr.update(visible=True)

    # Prevent registration with admin username
    if username == "admin":
        return "Error: Cannot create a user with the username 'admin'.", gr.update(visible=True)

    # Add new user to the database
    users[username] = {"password": password, "role": "user"}
    return f"User '{username}' registered successfully!", gr.update(visible=False)
def login(username, password):
    if username in users and users[username]["password"] == password:
        role = users[username]["role"]
        return (
            gr.update(visible=False),  # Hide login tab
            gr.update(visible=True),   # Show home page tab
            gr.update(visible=True),   # Show chatbot tab
            gr.update(visible=role == "admin"),  # Show analyze tab only for admin
            gr.update(visible=True),   # Show list of cars tab
            f"Welcome, {'Admin' if role == 'admin' else username}!",
            True,  # Logged in state
            role == "admin"  # Is admin
        )
    else:
        return (
            gr.update(visible=True),   # Keep login tab visible
            gr.update(visible=False),  # Hide home page tab
            gr.update(visible=False),  # Hide chatbot tab
            gr.update(visible=False),  # Hide analyze tab
            gr.update(visible=False),  # Hide list of cars tab
            "Invalid username or password",
            False,  # Not logged in state
            False   # Not admin
        )

def logout():
    return (
        gr.update(visible=True),   # Show login tab
        gr.update(visible=False),  # Hide home page tab
        gr.update(visible=False),  # Hide chatbot tab
        gr.update(visible=False),  # Hide analyze tab
        gr.update(visible=False),  # Hide list of cars tab
        "You have been logged out.",
        False,  # Not logged in state
        False   # Not admin
    )

def update_accumulated_data():
    accumulated_data = pd.concat(year_data.values(), ignore_index=True)
    return accumulated_data

def read_multiple_excel(selected_year, files):
    global year_data

    if not files:
        return gr.update(value=None), gr.update(value=None)

    all_dfs = []
    toyota_data = pd.DataFrame()  # Store filtered Toyota models for this upload
    for file in files:
        try:
            df = pd.read_excel(file.name, engine='openpyxl')
            df['Year'] = selected_year
            all_dfs.append(df)

            # Filter Toyota models for this file
            toyota_df = df[df['Model'].str.contains('Toyota', case=False, na=False)]
            toyota_data = pd.concat([toyota_data, toyota_df], ignore_index=True)
        except Exception as e:
            print(f"Error reading {file.name}: {str(e)}")

    # Update year_data for the selected year
    if not toyota_data.empty:
        year_data[selected_year] = pd.concat([year_data[selected_year], toyota_data], ignore_index=True)

    # Accumulate all years' data for global access
    accumulated_data = update_accumulated_data()

    return gr.update(value=toyota_data), gr.update(value=accumulated_data)

def display_data(selected_year):
    if selected_year == "All Years":
        return gr.update(value=update_accumulated_data())
    elif year_data[selected_year].empty:
        return gr.update(value=None)
    return gr.update(value=year_data[selected_year])

def confirm_finish():
    return gr.update(value="Data uploaded successfully.")

def load_extracted_data():
    accumulated_data = update_accumulated_data()
    status = f"Total number of Toyota cars: {len(accumulated_data)}"
    return status, accumulated_data
def filter_vehicle_type(selected_year, vehicle_type):
    # Handle "All Years" case
    if selected_year == "All Years":
        data = pd.concat(year_data.values(), ignore_index=True)  # Combine all years' data
    else:
        data = year_data.get(selected_year, pd.DataFrame())  # Get data for the selected year

    if data.empty:
        return f"No data available to filter for {vehicle_type}.", None

    try:
        # Check if the 'Veh Class' column exists
        if 'Veh Class' not in data.columns:
            return f"Error: 'Veh Class' column not found in the data.", None

        # Filter rows based on the vehicle type
        filtered_models = data[data['Veh Class'].str.contains(vehicle_type, case=False, na=False)]
        return f"Found {len(filtered_models)} {vehicle_type} models.", filtered_models

    except Exception as e:
        return f"An error occurred: {e}", None

# Specific filtering functions for vehicle types
def filter_midsize_car_data(file):
    return filter_vehicle_type(file, "midsize car")

def filter_minivan_data(file):
    return filter_vehicle_type(file, "minivan")

def filter_wagon_data(file):
    return filter_vehicle_type(file, "wagon")

def filter_standard_suv_data(file):
    return filter_vehicle_type(file, "standard suv")

def filter_small_suv_data(file):
    return filter_vehicle_type(file, "small suv")

# Define a custom theme
custom_theme = gr.themes.Soft(
    primary_hue=gr.themes.colors.red,
    secondary_hue=gr.themes.colors.red,
    neutral_hue=gr.themes.colors.slate,
)

# Create the Gradio interface with the custom theme
with gr.Blocks(theme=custom_theme) as demo:
    gr.Markdown("# Toyota Data Management System")

    login_state = gr.State(False)  # Track login state
    is_admin = gr.State(False)     # Track admin status

    # Tabs for Login and multiple features
    with gr.Tabs() as tabs:
        with gr.Tab("Login") as login_tab:
            username_input = gr.Textbox(label="Username", placeholder="Enter your username")
            password_input = gr.Textbox(label="Password", type="password", placeholder="Enter your password")
            login_button = gr.Button("Login", variant="primary")
            login_output = gr.Textbox(label="Result")


            gr.Markdown("## Register a New User")
            register_username = gr.Textbox(label="New Username", placeholder="Enter a new username")
            register_password = gr.Textbox(label="New Password", type="password", placeholder="Enter a new password")
            confirm_password = gr.Textbox(label="Confirm Password", type="password", placeholder="Confirm your password")
            register_button = gr.Button("Register", variant="secondary")
            register_output = gr.Textbox(label="Registration Status")

        # Home Page Tab
        with gr.Tab("Home Page", visible=False) as home_page_tab:
            gr.Markdown("## Welcome to the Home Page!")
            home_text = gr.Textbox(label="Selected Category", value="", interactive=False)

            # Shared status output and result table
            home_page_status_output = gr.Textbox(label="Status")
            home_page_result_table = gr.DataFrame(label="Filtered Models")

            year_dropdown = gr.Dropdown(choices=["All Years", "2021", "2022", "2023", "2024", "2025"], label="Select model")

            # Pickup filtering
            gr.Button("Show Pickup Models").click(
                fn=lambda selected_year: filter_vehicle_type(selected_year, "pickup"),
                inputs=[year_dropdown],  # Only pass the Gradio component
                outputs=[home_page_status_output, home_page_result_table]
            )

            # Small Car filtering
            gr.Button("Show small car Models").click(
                fn=lambda selected_year: filter_vehicle_type(selected_year, "small car"),
                inputs=[year_dropdown],  # Only pass the Gradio component
                outputs=[home_page_status_output, home_page_result_table]
            )

            # Midsize Car filtering
            gr.Button("Show midsize Models").click(
                fn=lambda selected_year: filter_vehicle_type(selected_year, "midsize car"),
                inputs=[year_dropdown],  # Only pass the Gradio component
                outputs=[home_page_status_output, home_page_result_table]
            )

            # Minivan filtering
            gr.Button("Show minivan Models").click(
                fn=lambda selected_year: filter_vehicle_type(selected_year, "minivan"),
                inputs=[year_dropdown],  # Only pass the Gradio component
                outputs=[home_page_status_output, home_page_result_table]
            )

            # Wagon filtering
            gr.Button("Show wagon Models").click(
                fn=lambda selected_year: filter_vehicle_type(selected_year, "station wagon"),
                inputs=[year_dropdown],  # Only pass the Gradio component
                outputs=[home_page_status_output, home_page_result_table]
            )

            # Standard SUV filtering
            gr.Button("Show standard SUV Models").click(
                fn=lambda selected_year: filter_vehicle_type(selected_year, "standard SUV"),
                inputs=[year_dropdown],  # Only pass the Gradio component
                outputs=[home_page_status_output, home_page_result_table]
            )

            # Small SUV filtering
            gr.Button("Show small SUV Models").click(
                fn=lambda selected_year: filter_vehicle_type(selected_year, "small SUV"),
                inputs=[year_dropdown],  # Only pass the Gradio component
                outputs=[home_page_status_output, home_page_result_table]
            )

            # Logout button
            logout_button = gr.Button("Logout", variant="primary")
            logout_button.click(fn=logout, inputs=[], outputs=[home_text, login_state])


       #Chatbot preprocessing
        auto_data = pd.read_csv('/content/sample_data/auto_data/regular_2021.csv')

        openai.api_key = "3494f8ff-7d8f-423c-8c7f-0d823e1e1ff6"
        openai.api_base = "https://api.sambanova.ai/v1"

        def query_openai(question):
              filtered_data = auto_data.head(50).to_json()
              response = openai.ChatCompletion.create(
                    model='Meta-Llama-3.1-8B-Instruct',
                    messages=[
                        {"role": "system", "content": "You are a helpful assistant."},
                        {"role": "user", "content": question},
                        {"role": "system", "content": f"Context: {filtered_data}"}
                    ],

                   temperature=0.1,
                    top_p=0.1
        )

              return response['choices'][0]['message']['content']


        # Gradio interface function
        def chatbot(question):
           return query_openai(question)


        # ChatBot Tab
        with gr.Tab("ChatBot", visible=False) as chat_tab:
            gr.Markdown("### Chatbot with CSV Data Context")

            with gr.Row():
                  question_box = gr.Textbox(label="Ask a question based on the data", placeholder="e.g., Which gasoline model has the highest Cmb MPG?")
                  submit_button = gr.Button("Submit")



            with gr.Row():
                  answer_box = gr.Textbox(label="Answer", interactive=False)

         # Link the button to the chatbot function
        submit_button.click(fn=chatbot, inputs=question_box, outputs=answer_box)



        # Analyze Data Tab (Admin only)
        with gr.Tab("Analyze", visible=False) as analyze_tab:
            gr.Markdown("#### Toyota Models Extractor (Admin Only)")
            gr.Markdown("Select a year and upload Excel files to view and accumulate Toyota models for that year.")

            with gr.Row():
                year_dropdown = gr.Dropdown(choices=["2021", "2022", "2023", "2024", "2025"], label="Select a year:")
                file_upload = gr.File(label="Upload Excel Files", file_count="multiple", file_types=[".xlsx", ".xls"])

            output_table = gr.Dataframe(label="Toyota Models")
            finish_button = gr.Button("Finish")
            confirmation = gr.Markdown(label="Confirmation")

            gr.Markdown("# View Accumulated Toyota Models")
            gr.Markdown("Select a year to view the accumulated Toyota models for that year, or select 'All Years' to view all data.")
            view_year_dropdown = gr.Dropdown(label="Select a year:", choices=["All Years", "2021", "2022", "2023", "2024", "2025"])
            view_output_table = gr.Dataframe(label="Accumulated Toyota Models")

        # List of Cars Tab
        with gr.Tab("List of Cars", visible=False) as list_of_cars_tab:
            gr.Markdown("### Total number of Toyota Cars")
            status_output = gr.Textbox(label="Status")
            result_table = gr.DataFrame(label="Filtered Toyota Cars")
            gr.Button("Load Extracted Data").click(
                fn=load_extracted_data,
                inputs=[],
                outputs=[status_output, result_table]
            )

    # Event handlers
    login_button.click(
        fn=login,
        inputs=[username_input, password_input],
        outputs=[login_tab, home_page_tab, chat_tab, analyze_tab, list_of_cars_tab, login_output, login_state, is_admin]
    )

    register_button.click(
        fn=register_user,
        inputs=[register_username, register_password, confirm_password],  #
        outputs=[register_output, register_username]
    )

    logout_button.click(
        fn=logout,
        outputs=[login_tab, home_page_tab, chat_tab, analyze_tab, list_of_cars_tab, login_output, login_state, is_admin]
    )

    file_upload.change(fn=read_multiple_excel, inputs=[year_dropdown, file_upload], outputs=[output_table, view_output_table])
    finish_button.click(fn=confirm_finish, outputs=confirmation)
    view_year_dropdown.change(fn=display_data, inputs=view_year_dropdown, outputs=view_output_table)

# Launch the interface
demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://63402a36b4a3e246a8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
